In [ ]:
!unzip my_model_final.zip

In [ ]:
!pip install bitsandbytes transformers accelerate


In [ ]:
from peft import PeftConfig, PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

base_model_name = "unsloth/mistral-7b-bnb-4bit"  # Path to your base model
adapter_model_name = "my_model_final"  # Path to your fine-tuned adapter model
max_output_lines = 10                             # Maximum number of output lines to show

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load base model and adapter
model = AutoModelForCausalLM.from_pretrained(base_model_name, device_map="auto")
model = PeftModel.from_pretrained(model, adapter_model_name)
model = model.bfloat16()
model = model.to(device)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)



In [ ]:
# Loop to continuously take user input and log input-output pairs
while True:
    input_text = input("Enter your prompt (or type 'exit' to quit): ")
    if input_text.lower() == "exit":
        break

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)
    outputs = model.generate(input_ids, max_new_tokens=1000)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    lines = generated_text.split('\n')[:max_output_lines]
    limited_text = '\n'.join(lines)

    print("Generated Output:")
    print(limited_text)
    print("=" * 50)  # Separator for readability

    # Log input and output to a file
    with open("conversation_log.txt", "a", encoding="utf-8") as log_file:
        log_file.write("User Input:\n")
        log_file.write(input_text + "\n\n")
        log_file.write("Model Output:\n")
        log_file.write(limited_text + "\n")
        log_file.write("=" * 50 + "\n\n")
